# NanoGPT - Math Trick

## Building manual intuition: averaging prior token features

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.set_printoptions(precision=1, sci_mode=False, profile='short')
generator = torch.manual_seed(42)

batch_size = 4
context_length = 6
vocab_size = 8 # 8 characters or language tokens possible

B,T,C = (batch_size, context_length, vocab_size)
B,T,C

(4, 6, 8)

In [ ]:
# x is our training data
x = torch.randn(batch_size,context_length, vocab_size)
x

tensor([[[ 0.48,  1.35, -0.16, -0.42,  0.94, -0.18,  1.06,  0.21],
         [ 1.31,  0.46,  0.26, -0.76, -2.05, -1.53,  0.40,  0.63],
         [-0.15, -2.32,  1.30,  0.49,  1.13, -0.36,  0.36,  2.00],
         [ 1.04,  1.69,  0.02, -0.83, -1.08, -0.78,  0.51,  0.08],
         [ 0.40,  1.99, -0.46, -0.06, -1.37,  0.33, -0.98,  0.30],
         [ 0.19,  0.41, -1.58,  2.25,  1.00,  1.36,  0.63,  0.41]],

        [[-0.35,  1.46,  0.17,  1.05,  0.01, -0.08,  0.64,  0.57],
         [ 0.51,  0.22, -0.91,  1.48, -0.91, -0.53, -0.81,  0.52],
         [-0.13,  0.78,  0.56,  1.86,  1.04, -0.86,  0.84, -0.32],
         [-1.98,  0.02, -1.41, -1.88, -0.18,  0.79,  0.52, -0.27],
         [ 1.71,  0.06,  0.86, -0.59, -1.03, -0.22,  0.80,  0.91],
         [ 0.27, -0.04, -0.48,  0.32,  0.39,  0.73,  0.25,  0.08]],

        [[-0.71, -0.05,  0.52,  0.97, -0.28, -0.61, -0.56, -0.97],
         [ 1.34,  0.71,  0.35, -0.54,  0.86, -0.67,  1.07, -0.25],
         [-2.31, -1.29,  0.21, -1.24,  1.86,  0.06,  0.77,

In [ ]:
# for the 2nd item in our batch (index 1), the features of token 5 (index 4) are:
x[1,4]

tensor([ 0.0, -0.3, -1.3, -0.6,  0.5,  0.5,  1.1,  0.1])

In [ ]:
# if instead we want all the features of all the tokens before token 5 (index 4):
x[1, :4]

tensor([[-0.9, -0.7,  0.1,  0.5, -0.5,  1.2, -0.8, -0.7],
        [-1.4,  0.0, -0.1,  0.7, -0.1,  1.8, -1.2,  1.4],
        [ 1.4,  0.9,  2.2,  0.5,  0.3, -0.2, -1.1,  1.3],
        [-0.2,  0.5,  0.1,  0.4,  0.6, -0.6, -2.2, -0.8]])

In [ ]:
# and if we want to include the features for token 5 itself:
x[1,:4+1]

tensor([[-0.9, -0.7,  0.1,  0.5, -0.5,  1.2, -0.8, -0.7],
        [-1.4,  0.0, -0.1,  0.7, -0.1,  1.8, -1.2,  1.4],
        [ 1.4,  0.9,  2.2,  0.5,  0.3, -0.2, -1.1,  1.3],
        [-0.2,  0.5,  0.1,  0.4,  0.6, -0.6, -2.2, -0.8],
        [ 0.0, -0.3, -1.3, -0.6,  0.5,  0.5,  1.1,  0.1]])

In [ ]:
# if we want to calculate the mean:
torch.mean(x[1, :4+1],0) # 0 here means: take means across the columns

tensor([-0.2,  0.1,  0.2,  0.3,  0.2,  0.5, -0.8,  0.2])

Note how the mean of `-0.9, 1.3, -0.7, -1.6, -1.4` is `-0.7`

In [ ]:
(-0.9+1.3-0.7-1.6-1.4)/5

-0.6599999999999999

In [ ]:
x_bag_of_words = torch.zeros((B,T,C))
x_bag_of_words

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
     

In [ ]:
for b in range(0,batch_size):
    for t in range(0,context_length):
        xprev_and_self = x[b,:t+1] # (t+1,vocab_size)
        x_bag_of_words[b,t] = torch.mean(xprev_and_self, 0) # (1, vocab_size)
x_bag_of_words

tensor([[[ 1.9,  1.5,  0.9, -2.1,  0.7, -1.2, -0.0, -1.6],
         [ 0.6,  1.6,  0.3, -1.8, -0.0, -0.9, -0.4, -0.4],
         [ 0.9,  1.0,  0.0, -1.0, -0.3, -0.2, -0.0,  0.3],
         [ 1.0,  1.1,  0.2, -0.4, -0.3, -0.2, -0.1,  0.4],
         [ 0.5,  0.7,  0.1, -0.0, -0.1, -0.2,  0.0,  0.2],
         [ 0.2,  0.7, -0.1, -0.1, -0.3,  0.2, -0.2,  0.1]],

        [[-0.9, -0.7,  0.1,  0.5, -0.5,  1.2, -0.8, -0.7],
         [-1.2, -0.3,  0.0,  0.6, -0.3,  1.5, -1.0,  0.3],
         [-0.3,  0.1,  0.7,  0.6, -0.1,  0.9, -1.0,  0.6],
         [-0.3,  0.2,  0.6,  0.5,  0.1,  0.5, -1.3,  0.3],
         [-0.2,  0.1,  0.2,  0.3,  0.2,  0.5, -0.8,  0.2],
         [-0.0, -0.0, -0.0,  0.4, -0.1,  0.3, -0.5,  0.3]],

        [[-2.5,  0.5,  0.8,  0.0,  0.6,  0.6,  1.1, -0.5],
         [-1.3,  0.6,  0.6,  0.1,  0.5,  0.9,  0.5, -0.4],
         [-1.4,  0.4,  0.2,  0.2,  0.5,  0.6,  0.2, -0.1],
         [-1.0,  0.3,  0.2,  0.4,  0.7,  0.6,  0.3,  0.0],
         [-0.4,  0.5, -0.1,  0.1,  0.5,  0.8,  0.4, 

Note how we calculated before that for:

- batch item 2 (index 1)
- token 5 (index 4)

The bag of words was: `[-0.7,  0.1, -0.6, -0.1, -0.1,  0.1, -0.0,  0.5]`

Each of these represent the mean of token 5 and all tokens that came before it, for each channel/features.

## Making this efficient

What we just did was incredibly inefficient and can be optimized through matrix multiplication.  Let's start by creating some test matrices to play with:

In [ ]:
a = torch.ones(3,3)
b = torch.randint(0,4,(3,2)).float() # need to convert to float for matrix multiplication
c = a @ b # matrix multiplication

a,b,c

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[0., 2.],
         [2., 1.],
         [1., 3.]]),
 tensor([[3., 6.],
         [3., 6.],
         [3., 6.]]))

Note how in the first row of matrix c, the `7 8` is the sum of columns `2 3 2` and `2 3 3` because of how matrix multiplying works.  The other rows are exactly the same values (`7 8`) because matrix a has `1` in every row.

Let's introduce the `torch.tril()` function now, which gives us back the lower triangle of a matrix, zero-ing out the upper part:

In [ ]:
a = torch.tril(a)
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

Let's multiply this by our matrix b:

In [ ]:
c = a @ b
a,b,c

(tensor([[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]]),
 tensor([[0., 2.],
         [2., 1.],
         [1., 3.]]),
 tensor([[0., 2.],
         [2., 3.],
         [3., 6.]]))

The first row of a is multiplied by both the columns of b to get to the first row of b:
`(1*2 + 0*3 + 0*2)=2   (1*2 + 0*3 + 0*3)=2` </br>
The second row of a is multiplied by both the columns of b to get to the second row of b:
`(1*2 + 1*3 + 0*2)=5   (1*2 + 1*3 + 0*3)=5` </br>
The third row of a is multiplied by both the columns of b to get to the third row of b:
`(1*2 + 1*3 + 1*2)=7   (1*2 + 1*3 + 1*3)=8`

Note how:

- the first row in the resulting matrix c has the original values of row 1 of b, 
- the second row has the sum of the values of row 1 and 2 of b and 
- the third row has the sum of the values of row 1, 2 and 3 of b

We don't want to sum however.  We want to try and get to a mean of a value and all values that came before it.  To get the mean, we need to divide the sum by the number of items.  For that reason, we won't use all `1`s to multiply with:

In [ ]:
a1 = torch.ones(3,3)
a2 = torch.tril(a1)
a2_sum = torch.sum(a2, 1, keepdim=True) # keepdim is required given that we need the broadcasting to work
a3 = a2 / a2_sum
a1, a2, a2_sum, a3, b

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]]),
 tensor([[1.],
         [2.],
         [3.]]),
 tensor([[1.0, 0.0, 0.0],
         [0.5, 0.5, 0.0],
         [0.3, 0.3, 0.3]]),
 tensor([[0., 2.],
         [2., 1.],
         [1., 3.]]))

If we now want the averages:

In [ ]:
c = a3 @ b
c

tensor([[0.0, 2.0],
        [1.0, 1.5],
        [1.0, 2.0]])

(see [Let's build GPT: from scratch, in code, spelled out.](https://youtu.be/kCc8FmEb1nY?t=3082) )

## Applying the matrix technique on our training data

Our training data `x` looked like:

In [ ]:
x # shape: (batch_size x context_length x vocab_size) == (B x T x C)

tensor([[[ 0.48,  1.35, -0.16, -0.42,  0.94, -0.18,  1.06,  0.21],
         [ 1.31,  0.46,  0.26, -0.76, -2.05, -1.53,  0.40,  0.63],
         [-0.15, -2.32,  1.30,  0.49,  1.13, -0.36,  0.36,  2.00],
         [ 1.04,  1.69,  0.02, -0.83, -1.08, -0.78,  0.51,  0.08],
         [ 0.40,  1.99, -0.46, -0.06, -1.37,  0.33, -0.98,  0.30],
         [ 0.19,  0.41, -1.58,  2.25,  1.00,  1.36,  0.63,  0.41]],

        [[-0.35,  1.46,  0.17,  1.05,  0.01, -0.08,  0.64,  0.57],
         [ 0.51,  0.22, -0.91,  1.48, -0.91, -0.53, -0.81,  0.52],
         [-0.13,  0.78,  0.56,  1.86,  1.04, -0.86,  0.84, -0.32],
         [-1.98,  0.02, -1.41, -1.88, -0.18,  0.79,  0.52, -0.27],
         [ 1.71,  0.06,  0.86, -0.59, -1.03, -0.22,  0.80,  0.91],
         [ 0.27, -0.04, -0.48,  0.32,  0.39,  0.73,  0.25,  0.08]],

        [[-0.71, -0.05,  0.52,  0.97, -0.28, -0.61, -0.56, -0.97],
         [ 1.34,  0.71,  0.35, -0.54,  0.86, -0.67,  1.07, -0.25],
         [-2.31, -1.29,  0.21, -1.24,  1.86,  0.06,  0.77,

It has 3 dimensions: batch, time/context, features.  Let's try to treat this matrix so that every value is the mean of all prior values _in the same batch item_.  So for example, the 3rd token (index 2) on the 2nd item in our batch (index 1) has features: <br/>
`[-0.1,  0.8,  0.6,  1.9,  1.0, -0.9,  0.8, -0.3]`

We want these features to be the averages of the features of itself and all the features of the prior tokens: 

- `[-0.4,  1.5,  0.2,  1.1,  0.0, -0.1,  0.6,  0.6]`
- `[ 0.5,  0.2, -0.9,  1.5, -0.9, -0.5, -0.8,  0.5]`

Let's start by applying a triangular vector (shape ) to each batch item:

In [ ]:
tril = torch.tril(torch.ones(T,T))
tril, tril.shape

(tensor([[1., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1.]]),
 torch.Size([6, 6]))

Let's apply this to our entire training data set; using broadcasting:

In [ ]:
tril @ x # element-wise (T,T) @ (BxTxC) == (BxTxT) @ (BxTxC) == (BxTxC)

tensor([[[ 1.9,  1.5,  0.9, -2.1,  0.7, -1.2, -0.0, -1.6],
         [ 1.2,  3.1,  0.5, -3.5, -0.0, -1.8, -0.8, -0.8],
         [ 2.8,  3.0,  0.0, -3.1, -0.8, -0.7, -0.0,  0.8],
         [ 4.1,  4.3,  0.6, -1.7, -1.0, -0.7, -0.3,  1.7],
         [ 2.7,  3.4,  0.4, -0.0, -0.7, -1.1,  0.0,  0.9],
         [ 1.2,  4.4, -0.5, -0.6, -2.0,  1.0, -1.2,  0.4]],

        [[-0.9, -0.7,  0.1,  0.5, -0.5,  1.2, -0.8, -0.7],
         [-2.3, -0.6,  0.0,  1.2, -0.6,  3.0, -2.0,  0.6],
         [-0.9,  0.2,  2.2,  1.7, -0.2,  2.8, -3.1,  1.9],
         [-1.0,  0.8,  2.3,  2.2,  0.3,  2.2, -5.3,  1.2],
         [-1.0,  0.4,  0.9,  1.6,  0.9,  2.7, -4.1,  1.2],
         [-0.3, -0.1, -0.1,  2.2, -0.9,  1.9, -2.8,  1.7]],

        [[-2.5,  0.5,  0.8,  0.0,  0.6,  0.6,  1.1, -0.5],
         [-2.7,  1.2,  1.2,  0.2,  0.9,  1.9,  1.1, -0.8],
         [-4.2,  1.1,  0.6,  0.7,  1.6,  1.9,  0.7, -0.2],
         [-4.2,  1.4,  0.7,  1.4,  2.7,  2.3,  1.4,  0.2],
         [-2.2,  2.4, -0.7,  0.3,  2.6,  3.9,  2.1, 

This is similar to our previous example on how to incrementally add numbers, but now with an additional broadcasted batch dimension.  We'll want not to add those numbers those, but to average them out as before:

In [ ]:
weights = tril / torch.sum(tril, 1, keepdim=True) # keepdim for broadcasting
weights

tensor([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.5, 0.5, 0.0, 0.0, 0.0, 0.0],
        [0.3, 0.3, 0.3, 0.0, 0.0, 0.0],
        [0.2, 0.2, 0.2, 0.2, 0.0, 0.0],
        [0.2, 0.2, 0.2, 0.2, 0.2, 0.0],
        [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]])

In [ ]:
x_bag_of_words_with_matrix = weights @ x
x[0], x_bag_of_words_with_matrix[0]

(tensor([[ 1.9,  1.5,  0.9, -2.1,  0.7, -1.2, -0.0, -1.6],
         [-0.8,  1.6, -0.4, -1.4, -0.7, -0.6, -0.8,  0.8],
         [ 1.6, -0.2, -0.5,  0.4, -0.8,  1.1,  0.8,  1.7],
         [ 1.3,  1.3,  0.6,  1.3, -0.2,  0.0, -0.3,  0.9],
         [-1.4, -0.9, -0.2,  1.7,  0.3, -0.4,  0.3, -0.8],
         [-1.6,  1.0, -0.9, -0.6, -1.3,  2.1, -1.2, -0.5]]),
 tensor([[ 1.9,  1.5,  0.9, -2.1,  0.7, -1.2, -0.0, -1.6],
         [ 0.6,  1.6,  0.3, -1.8, -0.0, -0.9, -0.4, -0.4],
         [ 0.9,  1.0,  0.0, -1.0, -0.3, -0.2, -0.0,  0.3],
         [ 1.0,  1.1,  0.2, -0.4, -0.3, -0.2, -0.1,  0.4],
         [ 0.5,  0.7,  0.1, -0.0, -0.1, -0.2,  0.0,  0.2],
         [ 0.2,  0.7, -0.1, -0.1, -0.3,  0.2, -0.2,  0.1]]))

In [ ]:
x_bag_of_words.allclose(x_bag_of_words_with_matrix)

True

In [ ]:
torch.set_printoptions(precision=2, sci_mode=False, profile='short')
print(x[0])
print('===')
print(x_bag_of_words_with_matrix[0])

tensor([[ 1.93,  1.49,  0.90, -2.11,  0.68, -1.23, -0.04, -1.60],
        [-0.75,  1.65, -0.39, -1.40, -0.73, -0.56, -0.77,  0.76],
        [ 1.64, -0.16, -0.50,  0.44, -0.76,  1.08,  0.80,  1.68],
        [ 1.28,  1.30,  0.61,  1.33, -0.23,  0.04, -0.25,  0.86],
        [-1.38, -0.87, -0.22,  1.72,  0.32, -0.42,  0.31, -0.77],
        [-1.56,  1.00, -0.88, -0.60, -1.27,  2.12, -1.23, -0.49]])
===
tensor([[ 1.93,  1.49,  0.90, -2.11,  0.68, -1.23, -0.04, -1.60],
        [ 0.59,  1.57,  0.25, -1.75, -0.02, -0.90, -0.41, -0.42],
        [ 0.94,  0.99,  0.00, -1.02, -0.27, -0.24, -0.00,  0.28],
        [ 1.02,  1.07,  0.16, -0.43, -0.26, -0.17, -0.07,  0.42],
        [ 0.54,  0.68,  0.08, -0.00, -0.14, -0.22,  0.01,  0.18],
        [ 0.19,  0.73, -0.08, -0.10, -0.33,  0.17, -0.20,  0.07]])


## Doing the same with SoftMax

This was our tril matrix, which we got by `tril = torch.tril(torch.ones(T,T))`

In [ ]:
tril

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [ ]:
weights = torch.zeros(T,T)
weights

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

We can mask the weights and and apply a number (here minus infinity) to the non-masked values:

In [ ]:
weights = weights.masked_fill(tril == 0, float('-inf')) # => this is essentially saying: the future cannot communicate with the past
weights

tensor([[0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0.]])

The above matrix is essentially making sure that for:

- the first token (which is line 1), it only has itself to refer to:   `[0., -inf, -inf, -inf, -inf, -inf]`
- token 2 (which is line 2), has itself and token 1 to refer to:       `[0., 0., -inf, -inf, -inf, -inf]`
- token 3 (which is line 3), has token 1, 2 and itself to refer to:    `[0., 0., 0., -inf, -inf, -inf]`

and so on...

In [ ]:
weights = weights.softmax(1)
weights

tensor([[1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.50, 0.50, 0.00, 0.00, 0.00, 0.00],
        [0.33, 0.33, 0.33, 0.00, 0.00, 0.00],
        [0.25, 0.25, 0.25, 0.25, 0.00, 0.00],
        [0.20, 0.20, 0.20, 0.20, 0.20, 0.00],
        [0.17, 0.17, 0.17, 0.17, 0.17, 0.17]])

In [ ]:
x_bag_of_words_using_softmax = weights @ x
torch.allclose(x_bag_of_words_using_softmax, x_bag_of_words)

True

(For more explanation, see [Let's build GPT: from scratch, in code, spelled out.](https://youtu.be/kCc8FmEb1nY?t=3355))

## Summary

What we've done here is to do a weighted average of elements and their past using matrix multiplication.  In our case here, all weights were equal, but you can see that this does not have to be the case necesarily.  The elements in the lower part of the triangular weights matrix determine how much of each element in the past the weighted average is build of.